In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash import Dash
from dash.dependencies import Input, Output, State
import base64

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pymongo import MongoClient
from bson.json_util import dumps

from crudProject import AnimalShelter



###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "snhuTom"

shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({})).drop(['_id', ''], axis = 1)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display' : 'none'}),
    html.A(
        href="www.snhu.edu",
        children = [
            html.Center(html.Img(alt="SNHU.edu", src='data:image/png;base64,{}'.format(encoded_image.decode()))),
        ]
    ),
    html.Center(html.B(html.H1('AUSTIN ANIMAL CENTER DATABASE'))),
    html.Hr(),
    html.Br(),
    html.Center(html.B(html.H3('DEVELOPED BY THOMAS COGLEY'))),
    html.Hr(),
    # Button location at top
    html.Div([
        html.Center(html.Label(['Select an option to filter the table by Animal type'], style={'font-weight': 'bold'})),
        html.Center(dcc.RadioItems(
        id = 'radio-items',
        options=[
            {'label' : 'Water Rescue', 'value': 'Water Rescue'},
            {'label' : 'Mountain Rescue', 'value' : 'Mountain Rescue'},
            {'label' : 'Disaster Rescue', 'value': 'Disaster Rescue'},
            {'label' : 'Reset', 'value': 'Reset'}
        ],
        value = 'Filter by Rescue Type',
        inputStyle={"margin-left": "20px"},
        labelStyle={'display': 'inline-block'}
        ))
    ]),
    html.Br(),
    
    dash_table.DataTable(
        id='datatable-interactivity',
        
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action='custom',
        sort_mode="multi",
        sort_by=[],
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
        
        html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex', "width": "500"},
             children=[
                 html.Div(
                     className='row',
                     style={'display': 'flex'},
                     children=[
                         # where the pie chart will go
                         dcc.Graph(
                             id='graph-id',
                         ),
                         # where the map will go
                         html.Div(
                             id='map-id',
                             className='col s12 m6',
                         ),
                     ]
                 ),
             ])
])
#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    Output('datatable-interactivity',"data"),
    [Input(component_id='radio-items', component_property="value")
    ])

def radioFilter(radio_options):
    # start case
    if radio_options == 'Water Rescue':
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'})).drop(['_id', ''], axis=1)
        dff = dff.loc[(dff['breed'] == 'Labrador Retriever Mix') |
                      (dff['breed'] == 'Chesapeake Bay Retriever') |
                      (dff['breed'] == 'Newfoundland')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Female')]
        dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156')
    elif radio_options == 'Mountain Rescue':
        dff = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'})).drop(['_id', ''], axis=1)
        dff = dff.loc[(dff['breed'] == 'German Shepherd') |
                      (dff['breed'] == 'Alaskan Malamute') |
                      (dff['breed'] == 'Old English Sheepdog') |
                      (dff['breed'] == 'Siberian Husky') |
                      (dff['breed'] == 'Rottweiler')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Male')]
        dff = dff.query('26 <= age_upon_outcome_in_weeks <= 156')
    elif radio_options == 'Disaster Rescue':
        dff = pd.DataFrame.from_records(shelter.read({})).drop(['_id', ''], axis=1)
        dff = dff.loc[(dff['breed'] == 'German Shepherd') |
                      (dff['breed'] == 'Doberman Pinscher') |
                      (dff['breed'] == 'Golden Retriever') |
                      (dff['breed'] == 'Bloodhound') |
                      (dff['breed'] == 'Rottweiler')]
        dff = dff.loc[(dff['sex_upon_outcome'] == 'Intact Male')]
        dff = dff.query('20 <= age_upon_outcome_in_weeks <= 300')
    elif radio_options == 'Reset':
        dff = pd.DataFrame.from_records(shelter.read({})).drop(['_id', ''], axis=1)
    else:
        dff = pd.DataFrame.from_records(shelter.read({})).drop(['_id', ''], axis=1)
      
    return dff.to_dict('records')

@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_rows')]
)

def highlightRows(selected_rows):
    """ highlight selected rows of dash table in Green """
    return [{
        'if': { 'row_index': i },
        'background_color': '#02FF3F'
    } for i in selected_rows]

@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-interactivity', "derived_virtual_data")])

def update_graph(allData):
    """ functionality for pie chart """
    dff = pd.DataFrame(allData)

    piechart = px.pie(
        data_frame=dff,
        # pie chart measures outcome_type column
        names=dff['outcome_type'],
        hole=.2,
    )
    return piechart

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
     Input('datatable-interactivity', 'derived_virtual_selected_rows')
     ])

def update_map(viewData, derived_virtual_selected_rows):
    """ functionality for updating map. map always shows location of the animal at top of table's current page """

    dff = df if viewData is None else pd.DataFrame(viewData)
    selected_animal = None

    # if there are no selected rows yet, map default displays first animal of table's current page
    if not derived_virtual_selected_rows:
        selected_animal = dff.iloc[0]
    # else there is a selected row, map displays that animal
    else:
        selected_animal = dff.iloc[derived_virtual_selected_rows[0]]

    latitude = selected_animal[12]
    longitude = selected_animal[13]
    animal_breed = selected_animal[3]
    animal_name = selected_animal[8]

    return [
        dl.Map(style={'width': '700px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[latitude, longitude], children=[
                     # show breed of animal on hovering over marker
              dl.Popup([
                dl.Tooltip(animal_breed),
                # show animal name on clicking marker
                 html.H1("Animal Name"),
                    html.P(animal_name)
                ])
            ])
        ])
    ]

# run application
app